# Chonkie FileFetcher - Complete Guide

This notebook demonstrates all possible combinations and usage patterns for the **FileFetcher** in Chonkie.

## What is FileFetcher?

FileFetcher retrieves files from your local filesystem for pipeline processing. It supports:
- **Single file mode**: Fetch a specific file by path
- **Directory mode**: Fetch multiple files from a directory
- **Extension filtering**: Filter files by extension when using directory mode

## Key Features:
- ✅ Fetch single files or entire directories
- ✅ Filter by file extensions (`.txt`, `.md`, `.pdf`, etc.)
- ✅ Works both in pipelines and standalone
- ✅ Clear error handling and validation

## Visual Overview

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#ff6b6b','primaryTextColor':'#fff','primaryBorderColor':'#c92a2a','lineColor':'#339af0','secondaryColor':'#51cf66','tertiaryColor':'#ffd43b','background':'#f8f9fa','mainBkg':'#e3fafc','secondBkg':'#fff3bf','tertiaryBkg':'#ffe3e3','textColor':'#212529','fontSize':'16px'}}}%%

graph TB
    Start([🎯 FileFetcher<br/>Entry Point]):::startClass
    
    Start --> Mode{Choose Mode}:::decisionClass
    
    Mode -->|path parameter| Single[📄 Single File Mode<br/>path='file.txt']:::singleClass
    Mode -->|dir parameter| Dir[📁 Directory Mode<br/>dir='./folder']:::dirClass
    
    Single --> SingleReturn[Returns: Path Object]:::returnClass
    
    Dir --> Filter{Apply Extension<br/>Filter?}:::decisionClass
    
    Filter -->|No filter| AllFiles[📦 Fetch All Files<br/>All extensions]:::allClass
    Filter -->|Single ext| OneExt["📝 Filter Single Type<br/>ext=.txt"]:::filterClass
    Filter -->|Multiple ext| MultiExt["🎯 Filter Multiple Types<br/>ext=.txt, .md"]:::filterClass
    
    AllFiles --> DirReturn[Returns: list of Path]:::returnClass
    OneExt --> DirReturn
    MultiExt --> DirReturn
    
    SingleReturn --> Usage{Usage Context}:::decisionClass
    DirReturn --> Usage
    
    Usage -->|Standalone| Standalone[🔧 Direct Usage<br/>fetcher.fetch&#40;...&#41;]:::standaloneClass
    Usage -->|Pipeline| Pipeline[⚡ Pipeline Integration<br/>.fetch_from&#40;'file', ...&#41;]:::pipelineClass
    
    Standalone --> StandaloneOut[Process Path Objects]:::outputClass
    Pipeline --> PipelineOut[Process Documents<br/>with Chunks]:::outputClass
    
    Start -.->|Error Cases| Errors[⚠️ Error Handling]:::errorClass
    Errors --> E1[❌ FileNotFoundError<br/>File doesn't exist]:::errorDetailClass
    Errors --> E2[❌ ValueError<br/>Both path & dir]:::errorDetailClass
    Errors --> E3[❌ ValueError<br/>Neither path nor dir]:::errorDetailClass
    
    classDef startClass fill:#4c6ef5,stroke:#364fc7,stroke-width:3px,color:#fff
    classDef decisionClass fill:#7950f2,stroke:#5f3dc4,stroke-width:2px,color:#fff
    classDef singleClass fill:#ff6b6b,stroke:#c92a2a,stroke-width:2px,color:#fff
    classDef dirClass fill:#fa5252,stroke:#e03131,stroke-width:2px,color:#fff
    classDef allClass fill:#51cf66,stroke:#37b24d,stroke-width:2px,color:#fff
    classDef filterClass fill:#20c997,stroke:#087f5b,stroke-width:2px,color:#fff
    classDef returnClass fill:#ffd43b,stroke:#fab005,stroke-width:2px,color:#333
    classDef standaloneClass fill:#ff922b,stroke:#e8590c,stroke-width:2px,color:#fff
    classDef pipelineClass fill:#748ffc,stroke:#4c6ef5,stroke-width:2px,color:#fff
    classDef outputClass fill:#69db7c,stroke:#40c057,stroke-width:2px,color:#fff
    classDef errorClass fill:#ff8787,stroke:#fa5252,stroke-width:3px,color:#fff
    classDef errorDetailClass fill:#ffc9c9,stroke:#ff6b6b,stroke-width:1px,color:#c92a2a
```

## Setup - Create Mock Files

First, let's create sample files to demonstrate the fetcher capabilities.

In [1]:
import os
import shutil
from pathlib import Path

# Create test directories
test_dir = Path("./test_fetcher_files")
subdirs = ["documents", "documents/subfolder", "mixed_files"]

# Clean up if exists
if test_dir.exists():
    shutil.rmtree(test_dir)

# Create directory structure
test_dir.mkdir(exist_ok=True)
for subdir in subdirs:
    (test_dir / subdir).mkdir(parents=True, exist_ok=True)

# Create sample text files
sample_texts = {
    "single_file.txt": "This is a standalone text file for testing single file mode.",
    "documents/doc1.txt": "Document 1 contains information about machine learning algorithms.",
    "documents/doc2.txt": "Document 2 discusses natural language processing techniques.",
    "documents/doc3.txt": "Document 3 explores computer vision applications.",
    "documents/subfolder/nested.txt": "This is a nested document in a subfolder.",
}

# Create sample markdown files
sample_markdown = {
    "documents/readme.md": "# README\n\nThis is a markdown readme file.",
    "documents/guide.md": "# Guide\n\n## Section 1\n\nDetailed guide content here.",
    "mixed_files/notes.md": "# Notes\n\n- Point 1\n- Point 2\n- Point 3",
}

# Create sample JSON files
sample_json = {
    "documents/data.json": '{"name": "test", "value": 123}',
    "mixed_files/config.json": '{"setting": "enabled", "count": 5}',
}

# Create sample Python files
sample_python = {
    "mixed_files/script.py": "def hello():\n    print('Hello, World!')\n",
}

# Write all files
all_files = {**sample_texts, **sample_markdown, **sample_json, **sample_python}
for file_path, content in all_files.items():
    full_path = test_dir / file_path
    full_path.write_text(content, encoding='utf-8')

print("✅ Created test directory structure:")
print(f"📁 {test_dir}/")
for root, dirs, files in os.walk(test_dir):
    level = root.replace(str(test_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}📁 {os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}📄 {file}")

✅ Created test directory structure:
📁 test_fetcher_files/
📁 test_fetcher_files/
  📄 single_file.txt
  📁 documents/
    📄 data.json
    📄 doc1.txt
    📄 doc2.txt
    📄 doc3.txt
    📄 guide.md
    📄 readme.md
    📁 subfolder/
      📄 nested.txt
  📁 mixed_files/
    📄 config.json
    📄 notes.md
    📄 script.py


## Installation

Make sure Chonkie is installed:

In [2]:
# Install chonkie if not already installed
# !pip install chonkie

# Import required modules
from chonkie import FileFetcher
from chonkie.pipeline import Pipeline

print("✅ Chonkie imported successfully!")

✅ Chonkie imported successfully!


---

## 1. Standalone Usage - Single File Mode

Fetch a single file by providing the `path` parameter.

In [3]:
fetcher = FileFetcher()

# Fetch a single file
single_file = fetcher.fetch(path="./test_fetcher_files/single_file.txt")

print("📄 Single File Fetched:")
print(f"  Type: {type(single_file)}")
print(f"  Path: {single_file}")
print(f"  Exists: {single_file.exists()}")
print(f"  Name: {single_file.name}")
print(f"  Suffix: {single_file.suffix}")
print(f"\n📖 Content:")
print(f"  {single_file.read_text()}")

📄 Single File Fetched:
  Type: <class 'pathlib.WindowsPath'>
  Path: test_fetcher_files\single_file.txt
  Exists: True
  Name: single_file.txt
  Suffix: .txt

📖 Content:
  This is a standalone text file for testing single file mode.


## 2. Standalone Usage - Directory Mode (All Files)

Fetch all files from a directory without filtering.

In [4]:
fetcher = FileFetcher()

# Fetch all files from a directory
all_files = fetcher.fetch(dir="./test_fetcher_files/documents")

print(f"📁 Fetched {len(all_files)} files from directory:")
for file in all_files:
    print(f"  📄 {file.name} ({file.suffix})")
    
print(f"\n📊 File Type Summary:")
from collections import Counter
extensions = Counter([f.suffix for f in all_files])
for ext, count in extensions.items():
    print(f"  {ext or 'no extension'}: {count} file(s)")

📁 Fetched 7 files from directory:
  📄 data.json (.json)
  📄 doc1.txt (.txt)
  📄 doc2.txt (.txt)
  📄 doc3.txt (.txt)
  📄 guide.md (.md)
  📄 readme.md (.md)
  📄 nested.txt (.txt)

📊 File Type Summary:
  .json: 1 file(s)
  .txt: 4 file(s)
  .md: 2 file(s)


## 3. Standalone Usage - Directory Mode with Single Extension Filter

Fetch only files with a specific extension (e.g., only `.txt` files).

In [5]:
fetcher = FileFetcher()

# Fetch only .txt files
txt_files = fetcher.fetch(dir="./test_fetcher_files/documents", ext=[".txt"])

print(f"📁 Fetched {len(txt_files)} .txt files:")
for file in txt_files:
    print(f"  📄 {file.name}")
    # Show a preview of content
    content = file.read_text()[:50]
    print(f"     Preview: {content}...")
    print()

📁 Fetched 4 .txt files:
  📄 doc1.txt
     Preview: Document 1 contains information about machine lear...

  📄 doc2.txt
     Preview: Document 2 discusses natural language processing t...

  📄 doc3.txt
     Preview: Document 3 explores computer vision applications....

  📄 nested.txt
     Preview: This is a nested document in a subfolder....



## 4. Standalone Usage - Directory Mode with Multiple Extension Filters

Fetch files with multiple extensions (e.g., `.txt` and `.md` files).

In [6]:
fetcher = FileFetcher()

# Fetch both .txt and .md files
text_files = fetcher.fetch(dir="./test_fetcher_files/documents", ext=[".txt", ".md"])

print(f"📁 Fetched {len(text_files)} text/markdown files:")
for file in text_files:
    print(f"  📄 {file.name} ({file.suffix})")

# Group by extension
txt_count = sum(1 for f in text_files if f.suffix == ".txt")
md_count = sum(1 for f in text_files if f.suffix == ".md")
print(f"\n📊 Breakdown:")
print(f"  .txt files: {txt_count}")
print(f"  .md files: {md_count}")

📁 Fetched 6 text/markdown files:
  📄 doc1.txt (.txt)
  📄 doc2.txt (.txt)
  📄 doc3.txt (.txt)
  📄 guide.md (.md)
  📄 readme.md (.md)
  📄 nested.txt (.txt)

📊 Breakdown:
  .txt files: 4
  .md files: 2


## 5. Standalone Usage - Mixed Directory with Various Extensions

Demonstrate fetching different file types from a directory with mixed content.

In [7]:
fetcher = FileFetcher()

# Example 1: Fetch only markdown files
md_files = fetcher.fetch(dir="./test_fetcher_files/mixed_files", ext=[".md"])
print(f"📝 Markdown files: {len(md_files)}")
for f in md_files:
    print(f"  - {f.name}")

# Example 2: Fetch only JSON files
json_files = fetcher.fetch(dir="./test_fetcher_files/mixed_files", ext=[".json"])
print(f"\n📦 JSON files: {len(json_files)}")
for f in json_files:
    print(f"  - {f.name}")

# Example 3: Fetch Python files
py_files = fetcher.fetch(dir="./test_fetcher_files/mixed_files", ext=[".py"])
print(f"\n🐍 Python files: {len(py_files)}")
for f in py_files:
    print(f"  - {f.name}")

# Example 4: Fetch multiple types at once
multi_files = fetcher.fetch(dir="./test_fetcher_files/mixed_files", ext=[".md", ".json", ".py"])
print(f"\n🎯 Multiple extensions (.md, .json, .py): {len(multi_files)} files")
for f in multi_files:
    print(f"  - {f.name} ({f.suffix})")

📝 Markdown files: 1
  - notes.md

📦 JSON files: 1
  - config.json

🐍 Python files: 1
  - script.py

🎯 Multiple extensions (.md, .json, .py): 3 files
  - config.json (.json)
  - notes.md (.md)
  - script.py (.py)


---

## 6. Pipeline Integration - Single File Mode

Use FileFetcher within a Chonkie pipeline to fetch, process, and chunk a single file.

In [8]:
# Pipeline with single file
doc = (Pipeline()
    .fetch_from("file", path="./test_fetcher_files/single_file.txt")
    .process_with("text")
    .chunk_with("recursive", chunk_size=30)  # Small chunk size for demo
    .run())

print(f"📄 Single File Pipeline Result:")
print(f"  Document: {doc.source if hasattr(doc, 'source') else 'N/A'}")
print(f"  Number of chunks: {len(doc.chunks)}")
print(f"\n📝 Chunks:")
for i, chunk in enumerate(doc.chunks, 1):
    print(f"  Chunk {i}: {chunk.text[:50]}...")

📄 Single File Pipeline Result:
  Document: N/A
  Number of chunks: 3

📝 Chunks:
  Chunk 1: This is a standalone text...
  Chunk 2:  file for testing single file...
  Chunk 3:  mode....


## 7. Pipeline Integration - Directory Mode (All Files)

Fetch all files from a directory and process them through a pipeline.

In [9]:
# Pipeline with directory (all files)
docs = (Pipeline()
    .fetch_from("file", dir="./test_fetcher_files/documents")
    .process_with("text")
    .chunk_with("recursive", chunk_size=50)
    .run())

print(f"📁 Directory Pipeline Result:")
print(f"  Processed {len(docs)} documents\n")

for i, doc in enumerate(docs, 1):
    source_name = Path(doc.source).name if hasattr(doc, 'source') else f"Document {i}"
    print(f"  {i}. {source_name}")
    print(f"     Chunks: {len(doc.chunks)}")
    if len(doc.chunks) > 0:
        print(f"     First chunk: {doc.chunks[0].text[:40]}...")
    print()

📁 Directory Pipeline Result:
  Processed 7 documents

  1. Document 1
     Chunks: 1
     First chunk: {"name": "test", "value": 123}...

  2. Document 2
     Chunks: 2
     First chunk: Document 1 contains information about ma...

  3. Document 3
     Chunks: 2
     First chunk: Document 2 discusses natural language pr...

  4. Document 4
     Chunks: 1
     First chunk: Document 3 explores computer vision appl...

  5. Document 5
     Chunks: 2
     First chunk: # Guide

## Section 1

Detailed guide co...

  6. Document 6
     Chunks: 1
     First chunk: # README

This is a markdown readme file...

  7. Document 7
     Chunks: 1
     First chunk: This is a nested document in a subfolder...



## 8. Pipeline Integration - Directory Mode with Extension Filter

Fetch only specific file types from a directory and process them.

In [10]:
# Pipeline with filtered extensions
docs = (Pipeline()
    .fetch_from("file", dir="./test_fetcher_files/documents", ext=[".txt", ".md"])
    .process_with("text")
    .chunk_with("recursive", chunk_size=50)
    .run())

print(f"📁 Filtered Pipeline Result (.txt and .md only):")
print(f"  Processed {len(docs)} documents\n")

# Group by extension
txt_docs = []
md_docs = []

for doc in docs:
    if hasattr(doc, 'source'):
        source_path = Path(doc.source)
        if source_path.suffix == ".txt":
            txt_docs.append(doc)
        elif source_path.suffix == ".md":
            md_docs.append(doc)

print(f"📄 Text files processed: {len(txt_docs)}")
for doc in txt_docs:
    print(f"   - {Path(doc.source).name}: {len(doc.chunks)} chunks")

print(f"\n📝 Markdown files processed: {len(md_docs)}")
for doc in md_docs:
    print(f"   - {Path(doc.source).name}: {len(doc.chunks)} chunks")

📁 Filtered Pipeline Result (.txt and .md only):
  Processed 6 documents

📄 Text files processed: 0

📝 Markdown files processed: 0


---

## 9. Error Handling - Invalid Path

In [11]:
fetcher = FileFetcher()

# Test 1: File not found
try:
    fetcher.fetch(path="./nonexistent_file.txt")
    print("✅ No error (unexpected)")
except FileNotFoundError as e:
    print(f"❌ FileNotFoundError (expected): {e}")
except Exception as e:
    print(f"❌ Other error: {type(e).__name__}: {e}")

❌ FileNotFoundError (expected): File not found: ./nonexistent_file.txt


## 10. Error Handling - Both Path and Dir Provided

In [12]:
fetcher = FileFetcher()

# Test 2: Both path and dir provided (should raise ValueError)
try:
    fetcher.fetch(path="./test_fetcher_files/single_file.txt", dir="./test_fetcher_files/documents")
    print("✅ No error (unexpected)")
except ValueError as e:
    print(f"❌ ValueError (expected): {e}")
except Exception as e:
    print(f"❌ Other error: {type(e).__name__}: {e}")

❌ ValueError (expected): Provide either 'path' or 'dir', not both


## 11. Error Handling - Neither Path nor Dir Provided

In [13]:
fetcher = FileFetcher()

# Test 3: Neither path nor dir provided (should raise ValueError)
try:
    fetcher.fetch()
    print("✅ No error (unexpected)")
except ValueError as e:
    print(f"❌ ValueError (expected): {e}")
except Exception as e:
    print(f"❌ Other error: {type(e).__name__}: {e}")

❌ ValueError (expected): Must provide either 'path' or 'dir'


---

## 12. Advanced Use Case - Processing Multiple Directories

In [14]:
fetcher = FileFetcher()

# Fetch from multiple directories and combine
directories = [
    "./test_fetcher_files/documents",
    "./test_fetcher_files/mixed_files"
]

all_files = []
for dir_path in directories:
    files = fetcher.fetch(dir=dir_path)
    all_files.extend(files)
    print(f"📁 {dir_path}: {len(files)} files")

print(f"\n📊 Total files from all directories: {len(all_files)}")
print("\n📋 All files:")
for file in all_files:
    print(f"  - {file.relative_to('./test_fetcher_files')} ({file.suffix})")

📁 ./test_fetcher_files/documents: 7 files
📁 ./test_fetcher_files/mixed_files: 3 files

📊 Total files from all directories: 10

📋 All files:
  - documents\data.json (.json)
  - documents\doc1.txt (.txt)
  - documents\doc2.txt (.txt)
  - documents\doc3.txt (.txt)
  - documents\guide.md (.md)
  - documents\readme.md (.md)
  - documents\subfolder\nested.txt (.txt)
  - mixed_files\config.json (.json)
  - mixed_files\notes.md (.md)
  - mixed_files\script.py (.py)


## 13. Advanced Use Case - Selective Processing by File Type

In [15]:
fetcher = FileFetcher()

# Example: Process text files differently than markdown files
txt_files = fetcher.fetch(dir="./test_fetcher_files/documents", ext=[".txt"])
md_files = fetcher.fetch(dir="./test_fetcher_files/documents", ext=[".md"])

print("🔍 Processing Strategy:\n")

print(f"📄 Text files ({len(txt_files)}): Use larger chunks for analysis")
for file in txt_files:
    content = file.read_text()
    print(f"   - {file.name}: {len(content)} characters")

print(f"\n📝 Markdown files ({len(md_files)}): Use smaller chunks for semantic search")
for file in md_files:
    content = file.read_text()
    # Count headings
    headings = content.count('#')
    print(f"   - {file.name}: {len(content)} characters, {headings} headings")

# Could process each type differently in pipelines
print("\n💡 Use Case: Process txt files with chunk_size=512, md files with chunk_size=256")

🔍 Processing Strategy:

📄 Text files (4): Use larger chunks for analysis
   - doc1.txt: 66 characters
   - doc2.txt: 60 characters
   - doc3.txt: 49 characters
   - nested.txt: 41 characters

📝 Markdown files (2): Use smaller chunks for semantic search
   - guide.md: 51 characters, 3 headings
   - readme.md: 41 characters, 1 headings

💡 Use Case: Process txt files with chunk_size=512, md files with chunk_size=256


## 14. Advanced Use Case - File Metadata Analysis

In [16]:
fetcher = FileFetcher()
import datetime

# Fetch all files and analyze metadata
all_files = fetcher.fetch(dir="./test_fetcher_files/documents")

print("📊 File Metadata Analysis:\n")
for file in all_files:
    # Get file stats
    stats = file.stat()
    size = stats.st_size
    modified_time = datetime.datetime.fromtimestamp(stats.st_mtime)
    
    print(f"📄 {file.name}")
    print(f"   Size: {size} bytes")
    print(f"   Type: {file.suffix}")
    print(f"   Modified: {modified_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   Path: {file}")
    print()

# Summary statistics
total_size = sum(f.stat().st_size for f in all_files)
print(f"📈 Summary:")
print(f"   Total files: {len(all_files)}")
print(f"   Total size: {total_size} bytes ({total_size/1024:.2f} KB)")

📊 File Metadata Analysis:

📄 data.json
   Size: 30 bytes
   Type: .json
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\data.json

📄 doc1.txt
   Size: 66 bytes
   Type: .txt
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\doc1.txt

📄 doc2.txt
   Size: 60 bytes
   Type: .txt
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\doc2.txt

📄 doc3.txt
   Size: 49 bytes
   Type: .txt
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\doc3.txt

📄 guide.md
   Size: 55 bytes
   Type: .md
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\guide.md

📄 readme.md
   Size: 43 bytes
   Type: .md
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\readme.md

📄 nested.txt
   Size: 41 bytes
   Type: .txt
   Modified: 2026-01-11 14:00:05
   Path: test_fetcher_files\documents\subfolder\nested.txt

📈 Summary:
   Total files: 7
   Total size: 344 bytes (0.34 KB)


---

## Summary: All FileFetcher Combinations

Here's a comprehensive reference of all possible usage patterns:

### **Standalone Usage**
| Mode | Parameters | Returns | Use Case |
|------|-----------|---------|----------|
| Single File | `path="file.txt"` | `Path` | Process one specific file |
| All Files in Dir | `dir="./folder"` | `list[Path]` | Process all files in directory |
| Filtered by Single Ext | `dir="./folder", ext=[".txt"]` | `list[Path]` | Process only .txt files |
| Filtered by Multiple Exts | `dir="./folder", ext=[".txt", ".md"]` | `list[Path]` | Process multiple file types |

### **Pipeline Integration**
| Mode | Pipeline Syntax | Output | Use Case |
|------|----------------|--------|----------|
| Single File | `.fetch_from("file", path="doc.txt")` | Single `Document` | Chunk one file |
| Directory | `.fetch_from("file", dir="./docs")` | `list[Document]` | Chunk all files |
| Filtered | `.fetch_from("file", dir="./docs", ext=[".txt"])` | `list[Document]` | Chunk filtered files |

### **Error Conditions**
- ❌ File not found: Raises `FileNotFoundError`
- ❌ Both `path` and `dir` provided: Raises `ValueError`
- ❌ Neither `path` nor `dir` provided: Raises `ValueError`

### **Best Practices**
- ✅ Use extension filtering for large directories to improve performance
- ✅ Use absolute paths when possible for clarity
- ✅ Handle errors appropriately with try-except blocks
- ✅ Process different file types with appropriate chunking strategies

## Cleanup

Remove the test files created for this demonstration.

In [17]:
# Clean up test files
import shutil

test_dir = Path("./test_fetcher_files")
if test_dir.exists():
    shutil.rmtree(test_dir)
    print("✅ Test files cleaned up successfully")
else:
    print("ℹ️ Test directory not found")

✅ Test files cleaned up successfully
